# 🎬 Movie Recommendation System

This notebook builds a **Content-Based Recommendation System** using TMDB 5000 Movie and Credits datasets.

In [ ]:
import pandas as pd
import numpy as np
import ast
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# Upload 'tmdb_5000_movies.csv' and 'tmdb_5000_credits.csv' via Colab or use file paths directly
movies = pd.read_csv("tmdb_5000_movies.csv")
credits = pd.read_csv("tmdb_5000_credits.csv")

# Merge datasets
movies = movies.merge(credits, on='title')
movies.shape

In [ ]:
# Keep only useful columns
movies = movies[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]
movies.head(2)

In [ ]:
def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

def get_director(obj):
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            return i['name']
    return ''

In [ ]:
movies.dropna(inplace=True)
movies['genres'] = movies['genres'].apply(convert)
movies['keywords'] = movies['keywords'].apply(convert)
movies['cast'] = movies['cast'].apply(lambda x: convert(x)[:3])
movies['crew'] = movies['crew'].apply(get_director)

In [ ]:
# Create a single string of tags
movies['overview'] = movies['overview'].apply(lambda x: x.split())
movies['crew'] = movies['crew'].apply(lambda x: [x])
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']
new_df = movies[['movie_id', 'title', 'tags']]
new_df['tags'] = new_df['tags'].apply(lambda x: " ".join(x))
new_df['tags'] = new_df['tags'].apply(lambda x: x.lower())
new_df.head()

In [ ]:
cv = CountVectorizer(max_features=5000, stop_words='english')
vectors = cv.fit_transform(new_df['tags']).toarray()

similarity = cosine_similarity(vectors)

In [ ]:
def recommend(movie):
    movie = movie.lower()
    if movie not in new_df['title'].str.lower().values:
        return f"Movie '{movie}' not found in database."
    index = new_df[new_df['title'].str.lower() == movie].index[0]
    distances = list(enumerate(similarity[index]))
    movies_list = sorted(distances, key=lambda x: x[1], reverse=True)[1:6]
    return [new_df.iloc[i[0]].title for i in movies_list]

In [ ]:
# Example
recommend("Avatar")

## 🛰️ Deployment Tips

- Upload this notebook and both CSV files to GitHub.
- Make sure to include a `README.md` and optionally convert this to a Streamlit app for deployment.